# Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Prepare inputs and targets

In [2]:
tesla_sentiment = pd.read_csv("sentiment_time_series/tesla_sentiment_time_series.csv")
tesla_sentiment.head()

,date,binary_mean,binary_squared_mean,binary_sqrt_mean,prob_mean,prob_squared_mean,prob_sqrt_mean
0,1/1/2020,0.0,0.0,0.0,0.0,0.0,0.0
1,1/2/2020,0.0,0.0,0.0,0.0,0.0,0.0
2,1/3/2020,0.0,0.0,0.0,0.0,0.0,0.0
3,1/4/2020,0.0,0.0,0.0,0.0,0.0,0.0
4,1/5/2020,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
tesla_lstm = pd.read_csv("TSLA_LSTM_predictions.csv")
tesla_lstm.head()

,Unnamed: 0,Date,Pred,Actual
0,2393,1/2/2020,83.771358,86.052002
1,2394,1/3/2020,86.065369,88.601997
2,2395,1/6/2020,88.515733,90.307999
3,2396,1/7/2020,90.103062,93.811996
4,2397,1/8/2020,93.680037,98.428001


In [4]:
merged = pd.merge(tesla_sentiment, tesla_lstm, left_on='date', right_on='Date')

In [5]:
merged_changed = merged[['date', 'prob_sqrt_mean', 'Pred', 'Actual']]
merged_changed = merged_changed.dropna()

In [6]:
inputs_1 = merged_changed.prob_sqrt_mean.values
inputs_1.shape

(86,)

In [7]:
inputs_2 = merged_changed.Pred.values
inputs_2.shape

(86,)

In [8]:
inputs = np.stack((inputs_1, inputs_2), axis=1)
inputs.shape

(86, 2)

In [9]:
targets = merged_changed.iloc[:, 3].values

# Fit a model

In [10]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
inputs = sc.fit_transform(inputs)

inputs = sm.add_constant(inputs)
model = sm.OLS(targets, inputs)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     9750.
Date:                Sat, 13 Feb 2021   Prob (F-statistic):           3.38e-99
Time:                        17:55:52   Log-Likelihood:                -303.20
No. Observations:                  86   AIC:                             612.4
Df Residuals:                      83   BIC:                             619.8
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        215.6231      0.902    238.960      0.0

# Save results

In [11]:
predictions = results.predict()

In [12]:
tesla_stock_prediction = pd.DataFrame().assign(Date=merged_changed['date'], Predictions=predictions, 
                                                Actual=merged_changed['Actual'])
tesla_stock_prediction.to_csv("tesla_stock_prediction_reddit.csv")